<a href="https://colab.research.google.com/github/maximofn/SuperFluidez/blob/master/Super_fluidez_DAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Super fluidez

Credits to original Colab file: 
https://github.com/baowenbo/DAIN/issues/44

Colab adaptation:
https://github.com/styler00dollar/DAIN

Enhancement by [Styler00Dollar](https://github.com/styler00dollar) aka "sudo rm -rf / --no-preserve-root#8353" on discord and [Alpha](https://github.com/AlphaGit). Please do not run this command in your linux terminal. It's rather meant as a joke.

Information obtained thanks to the [DotCSV](https://www.youtube.com/channel/UCy5znSnfMsDwaLlROnZ7Qbg) YouTube channel. From video: https://www.youtube.com/watch?v=rr7FjhhvEes&t=7s

A simple guide:
- Copy the .ipynb-file to your drive.
- Create a folder inside of Google Drive named "DAIN"
- Change the configurations in the next cell
- Run cells one by one

Stuff that should be improved:
- Alpha channel will be removed automatically and won't be added back. Anything related to alpha will be converted to black.
- Adding configuration to select speed
- Detect scenes to avoid interpolating scene-changes
- Auto-resume
- Copy `start_frame` - `end_frame` audio from original input to final output

## Comprobar GPU

In [1]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

# P100: 16GB (Works)
# T4: 16GB [RuntimeError: CUDA call failed]
# P4: 8GB (Works)
# K80: 8GB (Not tested)

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla K80, 418.67, 11441 MiB


Si no se tiene un modelo bueno de GPU se intenta que Google la cambie

In [0]:
a = []

while True:
  a.append(1)

## Configuracion

In [2]:
################# Configurations ############################

# Use the values in here to configure what you'd like DAIN to do.

# Input file: Path (relative to the root of your Google Drive) to the input file.
# For instance, if you save your "example.mkv" file in your Google Drive, inside a "videos" folder, the path would be:
# videos/example.mkv. Currenly videos and gifs are supported.
INPUT_FILENAME = "LidiaValentin12fps.mp4"
INPUT_FILEPATH = "Colab\ Notebooks/Remasterizacion\ de\ video/Super\ fluided/" + INPUT_FILENAME

# Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the
# filetype in the destination. MP4 is recommended.
OUTPUT_FILE_NAME = "LidiaValentin60fps.mp4"
OUTPUT_FILE_PATH = "Colab\ Notebooks/Remasterizacion\ de\ video/Super\ fluided/" + OUTPUT_FILE_NAME

################# Optional configurations ############################

# Target FPS = how many frames per second should the result have. This will determine how many intermediate images are
# interpolated.
TARGET_FPS = 60

# Frame input directly
# Use a path that is in your GDrive if you already have the list of frames in the format 00001.png, 00002.png, etc.
# Your GDrive is located at `/content/gdrive/My Drive/`
FRAME_INPUT_DIR = '/content/DAIN/input_frames'

# Frame output directory
# Use a location in your GDrive if you want the generated frames stored to your Google Drive.
# Your GDrive is located at `/content/gdrive/My Drive/DAIN/tmp`
FRAME_OUTPUT_DIR = '/content/DAIN/output_frames'

# Seamless playback
# Creates a seamless loop by using the first frame as last one as well. Set this to True this if loop is intended.
SEAMLESS = False

# Resize hotfix
# DAIN frames are a bit "shifted / smaller" compared to original input frames. This can partly be mitigated with resizing
# DAIN frames to the resolution +2px and cropping the result to the original resoultion with the starting point (1,1).
# Without this fix, DAIN tends to make "vibrating" output and it is pretty noticible with static elements like text.
# This hotfix tries to make such effects less visible for a smoother video playback. I do not know what DAINAPP
# uses as a fix for this problem, but the original does show such behaviour with the default test images. More advanced
# users can change the interpolation method. The methods cv2.INTER_CUBIC and cv2.INTER_LANCZOS4 are recommended.
# The current default value is cv2.INTER_LANCZOS4.
RESIZE_HOTFIX = True

# Auto-delete output PNG dir after ffmpeg video creation.
AUTO_REMOVE = False

INPUT_FILEPATH: Colab\ Notebooks/Remasterizacion\ de\ video/Super\ fluided/LidiaValentin12fps.mp4


## Instalación de dependencias

Esto puede tardar entre 15 y 20 minutos

In [4]:
from IPython.display import clear_output
!git clone https://github.com/styler00dollar/Colab-DAIN /content/DAIN

Cloning into '/content/DAIN'...
remote: Enumerating objects: 694, done.
remote: Total 694 (delta 0), reused 0 (delta 0), pack-reused 694
Receiving objects: 100% (694/694), 265.27 KiB | 879.00 KiB/s, done.
Resolving deltas: 100% (396/396), done.


In [5]:
# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd /content/DAIN/my_package/
!./build.sh
print("Building #1 done.")

/content/DAIN/my_package
Need pytorch>=1.0.0
./build.sh: line 4: activate: No such file or directory
running install
running bdist_egg
running egg_info
creating mindepthflowprojection_cuda.egg-info
writing mindepthflowprojection_cuda.egg-info/PKG-INFO
writing dependency_links to mindepthflowprojection_cuda.egg-info/dependency_links.txt
writing top-level names to mindepthflowprojection_cuda.egg-info/top_level.txt
writing manifest file 'mindepthflowprojection_cuda.egg-info/SOURCES.txt'
writing manifest file 'mindepthflowprojection_cuda.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'mindepthflowprojection_cuda' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/torch/include -I/usr/local/lib/python3.6/di

In [6]:
# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Building #2 done.")

/content/DAIN/PWCNet/correlation_package_pytorch1_0
Need pytorch>=1.0.0
./build.sh: line 4: activate: No such file or directory
running install
running bdist_egg
running egg_info
creating correlation_cuda.egg-info
writing correlation_cuda.egg-info/PKG-INFO
writing dependency_links to correlation_cuda.egg-info/dependency_links.txt
writing top-level names to correlation_cuda.egg-info/top_level.txt
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'correlation_cuda' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/torch/include -I/usr/local/lib/python3.6/dist-packages/torch/include/torch/csrc/api/include -

In [7]:
# Downloading pre-trained model
%cd /content/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

/content/DAIN
--2020-04-17 13:38:03--  http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
Resolving vllab1.ucmerced.edu (vllab1.ucmerced.edu)... 169.236.184.68
Connecting to vllab1.ucmerced.edu (vllab1.ucmerced.edu)|169.236.184.68|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96319643 (92M)
Saving to: ‘model_weights/best.pth’

model_weights/best. 100%[===================>]  91.86M  19.0MB/s    in 5.8s    

2020-04-17 13:38:09 (15.9 MB/s) - ‘model_weights/best.pth’ saved [96319643/96319643]



In [0]:
# Select GPU
!CUDA_VISIBLE_DEVICES=0

In [9]:
!sudo apt-get install imagemagick imagemagick-doc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  bc fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6-doc imagemagick-6.q16 javascript-common
  libcupsfilters1 libcupsimage2 libdjvulibre-text libdjvulibre21 libgs9
  libgs9-common libijs-0.35 libjbig2dec0 libjs-bootstrap libjs-jquery
  libjs-jquery-easing libjs-jquery-fancybox libjs-jquery-mousewheel liblqr-1-0
  libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra libmagickwand-6.q16-3
  libnetpbm10 libpulsedsp libwmf0.2-7 libxxf86dga1 netpbm poppler-data
  pulseaudio-utils wslu x11-utils
Suggested packages:
  fonts-noto ghostscript-x autotrace cups-bsd | lpr | lprng enscript gimp
  gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance sane-utils
  texlive-base-bin transfig ufraw-batch apache2 | lighttpd | httpd inkscape
  libjxr-tools libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho

## Detectar los FPS del video de entrada

In [10]:
# Detecting FPS of input file.
%shell yes | cp -f /content/drive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/

import os
filename = os.path.basename(INPUT_FILEPATH)

import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)

if(fps/TARGET_FPS>0.5):
  print("Define a higher fps, because there is not enough time for new frames. (Old FPS)/(New FPS) should be lower than 0.5. Interpolation will fail if you try.")

print ("fps = %0.4f" % fps)

fps = 12.0839


## Extraer todos los franes del video de entrada

In [11]:
# ffmpeg extract - Generating individual frame PNGs from the source file.
%cd /content/DAIN/
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'

%shell ffmpeg -i '/content/DAIN/{filename}' '{FRAME_INPUT_DIR}/%05d.png'

png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
clear_output()

pngs_generated_count = int(png_generated_count_command_result.output.strip())

import shutil
if SEAMLESS==True:
  pngs_generated_count += 1
  original = str(FRAME_INPUT_DIR)+"/00001.png"
  target = str(FRAME_INPUT_DIR)+"/"+str(pngs_generated_count).zfill(5)+".png"
  shutil.copyfile(original, target)

print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")

# Checking if PNG do have alpha
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
  print("Alpha detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

Input FPS: 12.083916083916083
144 frame PNGs generated.
/content/DAIN/input_frames
srgb detected


In [12]:
print(f"{pngs_generated_count} frame PNGs generated.")

144 frame PNGs generated.


In [0]:
!cp -r /content/DAIN/input_frames /content/drive/My\ Drive/Colab\ Notebooks/Remasterizacion\ de\ video/Super\ fluided

## Interpolacion de los frames

In [13]:
# Interpolation
%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN

!python colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {pngs_generated_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

/content/DAIN
revise the unique id to a random numer 81656
Namespace(SAVED_MODEL=None, alpha=[0.0, 1.0], arg='./model_weights/81656-Fri-Apr-17-13:44/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, end_frame=144, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, frame_input_dir='/content/DAIN/input_frames', frame_output_dir='/content/DAIN/output_frames', log='./model_weights/81656-Fri-Apr-17-13:44/log.txt', lr=0.002, netName='DAIN_slowmotion', no_date=False, numEpoch=100, occ_lr_coe=1.0, patience=5, rectify_lr=0.001, save_path='./model_weights/81656-Fri-Apr-17-13:44', save_which=1, seed=1, start_frame=1, time_step=0.20139860139860138, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
Interpolate 3 frames
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2506: User

In [0]:
!cp -r /content/DAIN/output_frames /content/drive/My\ Drive/Colab\ Notebooks/Remasterizacion\ de\ video/Super\ fluided

## Crear el video de salida

In [15]:
# Create video
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' '/content/DAIN/{OUTPUT_FILE_NAME}'
if(AUTO_REMOVE==True):
  !rm -rf {FRAME_OUTPUT_DIR}/*

/content/DAIN/output_frames
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-li

In [0]:
!cp /content/DAIN/LidiaValentin60fps.mp4 /content/drive/My\ Drive/Colab\ Notebooks/Remasterizacion\ de\ video/Super\ fluided

In [0]:
# [Experimental] Create video with sound
# Only run this, if the original had sound.
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -i '/content/DAIN/{filename}' -acodec copy output-audio.aac
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' -i output-audio.aac -shortest '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'
if(AUTO_REMOVE==True):
  !rm -rf {FRAME_OUTPUT_DIR}/*
  !rm -rf output-audio.aac

## Eliminar los frames generados

In [0]:
# Manually removing output PNG dir (before working with another file)
!rm -rf {FRAME_OUTPUT_DIR}/*
!rm -rf {FRAME_INTPUT_DIR}/*